In [1]:
# Dependencies
import requests
import json
import pandas as pd
import gmaps
from yelpapi import YelpAPI

# Google API Key
from config import google_key
from config import yelp_key
yelp_api = YelpAPI(yelp_key)

In [2]:
store_locations = pd.read_csv("Resources/projectbeggarsstart_geo.csv")
store_locations

,SQL_ID,Address,City,State,Latitude,Longitude
0,122,12700 S Kedzie,Blue Island,IL,41.661707,-87.700403
1,123,12660 S Western,Blue Island,IL,41.663078,-87.680465
2,10465,3277 W 115th St,Merrionette Park,IL,41.683913,-87.701136
3,11834,22149 Governors Hwy,Richton Park,IL,41.485875,-87.712412
4,10674,3524 Ridge Road,Lansing,IL,41.564555,-87.531317
5,10766,15600 S Cicero Road,Oak Forest,IL,41.608535,-87.737192
6,10783,650 W Lincoln Hwy,New Lenox,IL,41.518236,-87.981932
7,12294,11329 W. 143rd St.,Orland Park,IL,41.629047,-87.896268
8,12340,310 S Clinton St,Chicago,IL,41.877560,-87.641331
9,12896,165 Main St NW,Bourbonnais,IL,41.155617,-87.880621


In [3]:
# parameters
target_categories = "pizza"
target_radius = 3000
target_limit = 50

ID = []
name = []
rating = []
review_count = []
lat = []
lng = []
transactions = []
distance = []
category = []

for index, row in store_locations.iterrows():

    target_id = row["SQL_ID"]

    # Search Yelp
    search_results = yelp_api.search_query(latitude=row["Latitude"],longitude=row["Longitude"],categories=target_categories,radius=target_radius,limit=target_limit)

    for result in search_results["businesses"]:
        if "Beggar" not in result["name"]:
            ID.append(target_id)
            rating.append(result["rating"])
            review_count.append(result["review_count"])
            name.append(result["name"])
            lat.append(result["coordinates"]["latitude"])
            lng.append(result["coordinates"]["longitude"])
            transactions.append(result["transactions"])
            distance.append(result["distance"])
            category.append(result["categories"][0]["title"])

In [4]:
#print(json.dumps(places_data, indent=4, sort_keys=True))
#print(len(places_data["results"]))
places = pd.DataFrame({
    "ID" : ID,
    "Name" : name,
    "Category" : category,
    "Review Count" : review_count,
    "Rating" : rating,
    "Transactions" : transactions,
    "Distance" : distance,
    "Latitude" : lat,
    "Longitude" : lng
})
places.to_csv("Resources/competitors_yelp.csv", index=False)
places.head()

,ID,Name,Category,Review Count,Rating,Transactions,Distance,Latitude,Longitude
0,122,Milano's Pizza & Specialties,Pizza,91,4.5,[delivery],3956.489333,41.694444,-87.681053
1,122,Giuseppe's Pizza,Pizza,41,4.5,[],2091.769536,41.650014,-87.680680
2,122,Russo's Pizza,Pizza,30,4.5,[],2776.707536,41.681599,-87.720108
3,122,Bartolini's,Pizza,113,4.0,[],3845.343848,41.630159,-87.718085
4,122,S & T Provisions,Pizza,56,4.5,[],3553.092631,41.691590,-87.715860


In [5]:
gmaps.configure(api_key=google_key)
locations_beggars = store_locations[["Latitude", "Longitude"]].astype(float)
locations_others = places[["Latitude", "Longitude"]].astype(float)

In [6]:
figure_layout = {
    'width' : '600px',
    'height': '600px'
}

fig = gmaps.figure(layout=figure_layout)
beggars_layer = gmaps.symbol_layer(
    locations_beggars, fill_color='black', stroke_color='black', scale=3
)
others_layer = gmaps.symbol_layer(
    locations_others, fill_color='red', stroke_color='red', scale=2
)

fig.add_layer(others_layer)
fig.add_layer(beggars_layer)

fig

Figure(layout=FigureLayout(height='600px', width='600px'))